Task 1: Multi-layer ANN

Hyperparameters:
 1. Number of nodes
 2. Number of layers
 3. Activation function

In [1]:
# Load library
import pandas as pd
import numpy as np
import math
from enum import Enum
from sklearn.model_selection import train_test_split

In [2]:
# Load data
concrete = pd.read_csv('data/concrete_data.csv')
concrete.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
# Separate X and Y
# Then separate test and train set
# Also do the Cross-Validation (optional)
X = concrete.drop('concrete_compressive_strength', axis = 1)
y = concrete['concrete_compressive_strength']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 42)

In [4]:
# Activation function
def logistic(x):
    return 1/(1 + math.exp(-x))

def ReLU(x):
    return max(0, x)

def hyperbolic(x):
    return math.tanh(x)

class ActFunc(Enum):
    log = logistic
    relu = ReLU
    hb = hyperbolic

In [5]:
# Neural Network test!
from neuralNet import neuralNet
from layer import layer

network = neuralNet()

network.add(layer(ActFunc.relu,3))
#network.add(layer(ActFunc.relu,8))
#network.add(layer(ActFunc.hb, 10))
network.add(layer(ActFunc.relu,1))

Applying NN on the concrete data

In [8]:
# PSO test!
import pso

swarmsize = 10
alpha = 0.7
beta = 2
gamma = 1.5
delta = 1.5
epsilon = 0.4
n_iter = 10

particle_swarm_opti = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, delta, epsilon, n_iter)
opti_particle, best_mae_arr = particle_swarm_opti.optimise()



Current best mae: 1584.6943822212781
Current best mae: 1568.9383208090403
Current best mae: 944.9353016995076
Current best mae: 333.7351241971548
Current best mae: 49.30745838504819
Current best mae: 36.02132686084143
Current best mae: 34.63667850190363
Final best mae 34.63667850190363


In [9]:
best_mae_arr

[1584.6943822212781,
 1568.9383208090403,
 944.9353016995076,
 333.7351241971548,
 49.30745838504819,
 36.02132686084143,
 34.63667850190363]

In [ ]:
# Testing the optimisation
# Ooooo bad bad result :D
particle_swarm_opti.assessFitness(opti_particle, y_train)

30.33776128509392

In [ ]:
# Testing with the test set
weights, bias = particle_swarm_opti.assessFitness_helper(opti_particle) # Extract the optimised weight and bias
# Apply on the NN
y_pred = X_test.apply(network.forwardCalculation, args = (weights, bias), axis = 1)
mse = network.sseCalculation(y_pred, y_test)
mse

28.656285388374553

In [ ]:
# Potential ranges of PSO parameters to try
# Could try them like a gridsearch but its probably a bit too much for that, tuning one at a time is probably the way to go

alphaRange = np.arange(0.4,0.9,0.1)
betaRange = np.arange(1.5,2.5,0.1)
gammaRange = np.arange(1.5,2.5,0.1)
deltaRange = np.arange(1.5,2.5,0.1)
epsilonRange = np.arange(0.1,0.5,0.1) # not sure about the range of the learning rate, might need to experiment or look for more sources